In [1]:
from pathgen.utils.seeds import set_seed
from pathgen.utils.paths import project_root

experiment_name = "one"
experiment_root = project_root() / "experiments" / experiment_name

global_seed = 123
set_seed(global_seed)

In [2]:
from pl_bolts.models.gans.dcgan.dcgan_module import DCGAN
from torchvision.datasets import ImageFolder
from torchvision import transforms as transform_lib

transforms = transform_lib.Compose([
    transform_lib.Resize(256),
    transform_lib.ToTensor(),
    transform_lib.Normalize((0.5, ), (0.5, )),
])

train_set = ImageFolder(experiment_root / "gan_training_patches", transform=transforms)
len(train_set)



14001

In [ ]:
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pl_bolts.callbacks import LatentDimInterpolator, TensorboardGenerativeModelImageSampler
from pytorch_lightning import loggers as pl_loggers

model = DCGAN(image_channels=3)

callbacks = [
    TensorboardGenerativeModelImageSampler(num_samples=5),
    LatentDimInterpolator(interpolate_epoch_interval=5),
]

dataloader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=64)

csv_logger = pl_loggers.CSVLogger(experiment_root / 'logs', name='dcgan', version=0)

trainer = pl.Trainer(callbacks=callbacks, logger=csv_logger, log_every_n_steps=1)
trainer.fit(model, dataloader)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/ubuntu/anaconda3/envs/pathgen/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory /home/ubuntu/pathgen/experiments/one/logs/dcgan/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/8
